# M203 Electronic markets project

Marchessaux François, Collin Thibault

## Exercise 1 - Target Close and TWAP orders

### Loading libraries and initializing parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
# Initialize the fixed parameters for inventory, risk aversion, market impact and volatility
x0 = 10e4
fixed_lambda = 0.1
fixed_eta = 1 * 1e-4
sigma = 0.2
T = 1

# Initialize the varying parameters
t_values = np.linspace(0, T, 1000)
lambdas = [1e-3, 1e-2 , 1e-1, 1, 3, 8]
etas = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]

### Define functions

In [ ]:
def compute_optimal_strategy(order_type, t, x0, sigma, lamb, eta, T):
  """
  Computes optimal deterministic strategy at a given time and for given parameters for different type of orders

  order_type: type of the order (IS, TC or TWAP)
  t: current time
  x0: initial inventory
  sigma: underlying spot volatility
  lamb: risk aversion parameter
  eta: multiplicative factor for temporary market impact
  T: end date of the execution process
  """
  result = 0
  k = sigma * np.sqrt(lamb / eta)

  if order_type == "IS":
    result = (x0 / np.sinh(k * T)) * np.sinh(k * (T - t))
  elif order_type == "TC":
    result = x0 * (1 - np.sinh(k * t) / np.sinh(k * T))
  elif order_type == "TWAP":
    result = x0 * (1 - t / T)

  return result

def compute_strategies(order_type, varying_param, varying_values, fixed_value):
  """
  Computes the optimal strategies for varying parameters

  order_type: type of the order (IS, TC or TWAP)
  varying_param: name of the varying parameter
  varying_values: values of the varying parameter
  fixed_value: value of the fixed parameter
  """

  df_list = []

  # Check which parameter is varying
  flag = varying_param == "lambda"

  # Compute the optimal strategy for each varying parameter
  for value in varying_values:
      xt = [compute_optimal_strategy(order_type, t, x0, sigma, value if flag else fixed_value, fixed_value if flag else value, T) for t in t_values]
      df_list.append(pd.DataFrame({'t': t_values, 'x': xt, varying_param: str(value)}))

  return pd.concat(df_list, ignore_index=True)


def plot_strategy(df, color, title):
  """
  Display the computed strategies

  df: dataframe containing the optimal strategies
  color: name of the varying parameter for the legend
  title: title of legend
  """

  fig = px.line(df, x='t', y='x', color=color, template='plotly_white',
                labels={"x": "Remaining Inventory", "t": "Time", color: title})
  fig.update_traces(hoverinfo='skip')
  fig.update_traces(hovertemplate=None)
  fig.update_layout(width=900, height=300)
  fig.show()

def compute_expectation(order_type, k, sigma, eta, x0, T):
  """
  Computes the expectation of the given order type, given the closed-form solved formula

  order_type: type of the order (IS, TC or TWAP)
  t: current time
  x0: initial inventory
  sigma: underlying spot volatility
  lamb: risk aversion parameter
  eta: multiplicative factor for temporary market impact
  T: end date of the execution process
  """
  result = 0

  if order_type == "IS":
    result = -((((k * x0) / (np.sinh(k * T)))**2) * eta) * ((2 * k * T + np.sinh(2 * k * T)) / (4 * k))
  elif order_type == "TC":
    result = -((((k * x0) / (np.sinh(k * T)))**2) * eta) * ((2 * k * T + np.sinh(2 * k * T)) / (4 * k))
  elif order_type == "TWAP":
    result = -(eta * (x0 ** 2) / T)
  else:
    print("Wrong order type")

  return result

def compute_variance(order_type, k, sigma, eta, x0, T):
  """
  Computes the variance of the given order type, given the closed-form solved formula

  order_type: type of the order (IS, TC or TWAP)
  t: current time
  x0: initial inventory
  sigma: underlying spot volatility
  lamb: risk aversion parameter
  eta: multiplicative factor for temporary market impact
  T: end date of the execution process
  """
  result = 0

  if order_type == "IS":
    result = (((sigma * x0) / (np.sinh(k * T)))**2) * ((np.sinh(2 * k * T) - (2 * k * T)) / (4 * k))
  elif order_type == "TC":
    result = (((sigma * x0) / (np.sinh(k * T)))**2) * ((np.sinh(2 * k * T) - (2 * k * T)) / (4 * k))
  elif order_type == "TWAP":
    result = 0
  else:
    print("Wrong order type")

  return result

def display_efficient_frontier(expectation, variance):
  # Store and scale the results
  df = pd.DataFrame({'E': np.abs(expectation) / 100, 'V': np.abs(variance) / 10000, 'lambda': lbd_values})

  # Display the efficient frontier
  fig = px.scatter(df, x='V', y='E', color='lambda', template='plotly_white',
                  labels={"V": "Variance", "E": "Abs(Expectation)", "lambda": "𝛌"}, color_continuous_scale='Purp')
  fig.update_layout(width=900, height=300)
  fig.show()

def Bellman(order_type, lamb, sigma, eta, nb_stocks, nb_periods):
    # Initialize matrices to store rewards and policies for each stock count and period
    reward = np.zeros((nb_stocks+1, nb_periods))
    policy = np.zeros((nb_stocks+1, nb_periods))

    # Iterate through each period in reverse order. This is because the decision at any given time
    # depends on the outcomes of future decisions
    for period in range(nb_periods-1, -1, -1):
        # Iterate through each possible number of stocks
        for stock in range(nb_stocks+1):
            psi = stock  # Current stock holding
            x = nb_stocks - psi  # Calculate the number of stocks not currently held

            # Terminal condition: If all stocks are held, no further action is taken
            if psi == nb_stocks:
                reward[psi, period] = 0
                policy[psi, period] = 0

            else:
                # Terminal period: Set the reward to infinity to highly penalize this situation
                if period == nb_periods - 1:
                    reward[psi, period] = np.inf
                    policy[psi, period] = np.inf
                # Penultimate period: Calculate the immediate reward based on holding and not holding stocks
                elif period == nb_periods - 2:
                    reward[psi, period] = -lamb * (sigma ** 2) * (x ** 2) - eta * (x ** 2) if order_type == "IS" else -lamb * (sigma ** 2) * ((-psi) ** 2) - eta * (x ** 2)
                    policy[psi, period] = x

                else:
                    # For all other periods, calculate the reward considering the current and future benefits
                    base_reward_calculation = -lamb * sigma * (x ** 2) if order_type == "IS" else -lamb * sigma * ((x - nb_stocks) ** 2)
                    r = base_reward_calculation + reward[psi, period+1]
                    p = 0

                    # Consider all other liquidation scenarios
                    i = 1
                    while (x - i >= 0):
                        # Compute the potential reward with the new liquidation quantity
                        potential_r = base_reward_calculation + reward[psi + i, period + 1] - eta * (i ** 2)

                        # If new liquidation strategy is more optimal, keep it
                        if potential_r > r :
                            r = potential_r
                            p = i

                        i += 1

                    # Fill the grid with the computed reward and policy
                    reward[psi, period] = r
                    policy[psi, period] = p

    # Determine the optimal trajectory of stock holdings over time, initializing the strategy at inception
    trajectory = [nb_stocks]
    initial = 0

    # Calculate the optimal path of holding stocks over each period
    for period in range(1, nb_periods + 1):
        instant_past = trajectory[-1]
        psi = int(policy[initial, period - 1])
        initial += psi
        new_remaining_inventory = instant_past - psi
        trajectory.append(new_remaining_inventory)

    return trajectory

### Implementation Shortfall - Optimal Strategies

#### Varying Risk Aversion

In [ ]:
df_IS_lambda = compute_strategies("IS", "lambda", lambdas, fixed_eta)
plot_strategy(df_IS_lambda, "lambda", "Risk Aversion")

#### Varying Temporary Market Impact

In [ ]:
df_IS_eta = compute_strategies("IS", "eta", etas, fixed_lambda)
plot_strategy(df_IS_eta, "eta", "Market Impact")

### Target Close - optimal strategy

#### Varying Risk Aversion

In [ ]:
df_TC_lambda = compute_strategies("TC", "lambda", lambdas, fixed_eta)
plot_strategy(df_TC_lambda, "lambda", "Risk Aversion")

#### Varying Temporary Market Impact

In [ ]:
df_TC_eta = compute_strategies("TC", "eta", etas, fixed_lambda)
plot_strategy(df_TC_eta, "eta", "Market Impact")

### TWAP - optimal strategy

#### Varying Risk Aversion

In [ ]:
df_TWAP_lambda = compute_strategies("TWAP", "lambda", lambdas, fixed_eta)
plot_strategy(df_TWAP_lambda, "lambda", "Risk Aversion")

#### Varying Temporary Market Impact

In [ ]:
df_TWAP_eta = compute_strategies("TWAP", "eta", etas, fixed_lambda)
plot_strategy(df_TWAP_eta, "eta", "Market Impact")

### Implementation Shortfall - Efficient Frontier

In [ ]:
# Compute the values of k given the different levels of risk aversion
lbd_values = np.linspace(1e-4, 1, num=1000)
k_values = np.sqrt(lbd_values / fixed_eta) * sigma

# Compute the variance and the expectation
IS_expectation = compute_expectation("IS", k_values, sigma, fixed_eta, x0, T)
IS_variance = compute_variance("IS", k_values, sigma, fixed_eta, x0, T)

display_efficient_frontier(IS_expectation, IS_variance)

### Target Close - Efficient Frontier

In [ ]:
# Compute the variance and the expectation
TC_expectation = compute_expectation("TC", k_values, sigma, fixed_eta, x0, T)
TC_variance = compute_variance("TC", k_values, sigma, fixed_eta, x0, T)

display_efficient_frontier(TC_expectation, TC_variance)

### TWAP - Efficient Frontier

In [ ]:
# Compute the variance and the expectation
TWAP_expectation = compute_expectation("TWAP", k_values, sigma, fixed_eta, x0, T)
TWAP_variance = compute_variance("TWAP", k_values, sigma, fixed_eta, x0, T)

display_efficient_frontier(TC_expectation, TC_variance)

### Bellman Optimization

In [ ]:
# Initializing the parameters
bellman_x0 = 200
sigma = 0.20
bellman_eta = 5 * 1e-1
steps = 150
lambdas = [5e-4, 1e-3, 1e-2 , 1e-1, 1, 10]

#### Implementation Shortfall

In [ ]:
tab = {'t':[],'x':[],'lambda':[]}

# Compute the optimal strategies for each level of risk aversion
for lamb_value in lambdas:
    traj = Bellman("IS", lamb_value, sigma, bellman_eta, bellman_x0, steps)
    tab['t'].extend(range(len(traj)))
    tab['x'].extend(traj)
    tab['lambda'].extend([str(lamb_value) for i in range(len(traj))])

df = pd.DataFrame(tab)

plot_strategy(df, 'lambda', 'Risk Aversion')

#### Target Close

In [ ]:
tab = {'t':[],'x':[],'lambda':[]}

# Compute the optimal strategies for each level of risk aversion
for lamb_value in lambdas:
    traj = Bellman("TC", lamb_value, sigma, bellman_eta, bellman_x0, steps)
    tab['t'].extend(range(len(traj)))
    tab['x'].extend(traj)
    tab['lambda'].extend([str(lamb_value) for i in range(len(traj))])

df = pd.DataFrame(tab)

plot_strategy(df, 'lambda', 'Risk Aversion')